In [1]:
import pandas as pd
import ast
import numpy as np
from tqdm import tqdm

/var/folders/00/ft6n0snn3g5gtsxj0z9n8lhh0000gp/T/ipykernel_20564/2960853757.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [25]:
df = pd.read_csv("results_v2.csv")

In [26]:
df.head()

,filename,node_name,tags,graph_embedding,text,label,embedded_texts
0,cleaneval/prepared_html/ce_1.html,span_0,"{'div': 0, 'center': 0, 'table': 0, 'tbody': 0...",[ 2.66596258e-01 2.19546929e-02 1.64745390e-...,Las Vegas Realtor - House Sell - Sales - Real ...,0,"[-0.05825778469443321, -0.05459720641374588, -..."
1,cleaneval/prepared_html/ce_1.html,span_1,"{'div': 0, 'center': 0, 'table': 1, 'tbody': 1...",[ 0.43698856 -0.47383237 0.22595443 0.004418...,Home,0,"[-0.08070644736289978, -0.043665092438459396, ..."
2,cleaneval/prepared_html/ce_1.html,span_2,"{'div': 0, 'center': 0, 'table': 1, 'tbody': 1...",[ 0.4270207 -0.54548746 0.23394041 -0.111042...,|,0,"[-0.046702198684215546, -0.02059364505112171, ..."
3,cleaneval/prepared_html/ce_1.html,span_3,"{'div': 0, 'center': 0, 'table': 1, 'tbody': 1...",[ 0.78561485 -0.698475 0.44287145 -0.314255...,Current House Value,0,"[-0.026915384456515312, -0.07351666688919067, ..."
4,cleaneval/prepared_html/ce_1.html,span_4,"{'div': 0, 'center': 0, 'table': 1, 'tbody': 1...",[ 0.4832072 -0.5486914 0.2184332 -0.156697...,|,0,"[-0.046702198684215546, -0.02059364505112171, ..."


In [27]:
def get_tags(tags_string):
    tags_dict = ast.literal_eval(tags_string)
    res = []
    for val in tags_dict.values():
        res.append(val)
    return res

In [28]:
def get_array(array_string):
    array = ast.literal_eval(array_string)
    return array

In [50]:
def get_graph_embedding(array_string):
    res = []
    for p in array_string.replace("]", " ").replace("[", " ").split():
        if len(p) > 0:
            try:
                f = float(p)
                res.append(f)
            except:
                print("baj van")
                continue
    if len(res) != 64:
        print("Nagy baj van")
    return res

In [30]:
def get_nr_file(filename):
    return filename.split("_")[-1].split(".")[0]

In [31]:
def get_node_nr(node):
    return node.split("_")[1]

In [32]:
len(get_tags(df.iloc[1]['tags']))

64

In [51]:
len(get_graph_embedding(df.iloc[1]['graph_embedding']))

64

In [34]:
len(get_array(df.iloc[1]['embedded_texts']))

512

In [35]:
get_nr_file(df.iloc[1]['filename'])

'1'

In [36]:
get_node_nr(df.iloc[2]['node_name'])

'2'

In [52]:
final_df = []
lengths = []

for _, row in tqdm(df.iterrows()):
    nr_file = get_nr_file(row['filename'])
    nr_node = get_node_nr(row['node_name'])
    tags = get_tags(row['tags'])
    embedded_graph = get_graph_embedding(row['graph_embedding'])
    embedded_texts = get_array(row['embedded_texts'])
    final_df.append([nr_file, nr_node] + tags + embedded_graph + embedded_texts)
    lengths.append([len(tags), len(embedded_graph), len(embedded_texts)])


143363it [03:09, 754.63it/s] 


In [53]:
len(lengths)

143363

In [54]:
len(final_df)

143363

In [62]:
final_df = pd.DataFrame(final_df)

In [55]:
lengths_array = np.asarray(lengths)

In [56]:
lengths_array

array([[ 64,  64, 512],
       [ 64,  64, 512],
       [ 64,  64, 512],
       ...,
       [ 64,  64, 512],
       [ 64,  64, 512],
       [ 64,  64, 512]])

In [60]:
np.sum(lengths_array[:, 1]) / 64

143363.0

In [63]:
final_df.head()

,0,1,2,3,4,5,6,7,8,9,...,632,633,634,635,636,637,638,639,640,641
0,1,0,0,0,0,0,0,0,0,0,...,-0.000592,0.040750,0.023374,-0.009162,0.054658,0.067103,-0.041042,-0.054814,0.058525,-0.004794
1,1,1,0,0,1,1,2,2,1,1,...,0.053507,-0.099840,0.004646,0.058159,-0.028178,-0.007119,0.028393,-0.038878,0.061012,-0.012936
2,1,2,0,0,1,1,2,2,1,1,...,0.055856,-0.041606,0.034454,0.060111,-0.052395,0.068730,-0.012220,-0.054667,0.000948,0.034500
3,1,3,0,0,1,1,2,2,1,1,...,0.045155,-0.094596,-0.066634,0.036554,0.026686,-0.024640,0.034905,-0.044435,0.016929,0.040808
4,1,4,0,0,1,1,2,2,1,1,...,0.055856,-0.041606,0.034454,0.060111,-0.052395,0.068730,-0.012220,-0.054667,0.000948,0.034500


In [64]:
final_df['label'] = df['label']

In [65]:
final_df.head()

,0,1,2,3,4,5,6,7,8,9,...,633,634,635,636,637,638,639,640,641,label
0,1,0,0,0,0,0,0,0,0,0,...,0.040750,0.023374,-0.009162,0.054658,0.067103,-0.041042,-0.054814,0.058525,-0.004794,0
1,1,1,0,0,1,1,2,2,1,1,...,-0.099840,0.004646,0.058159,-0.028178,-0.007119,0.028393,-0.038878,0.061012,-0.012936,0
2,1,2,0,0,1,1,2,2,1,1,...,-0.041606,0.034454,0.060111,-0.052395,0.068730,-0.012220,-0.054667,0.000948,0.034500,0
3,1,3,0,0,1,1,2,2,1,1,...,-0.094596,-0.066634,0.036554,0.026686,-0.024640,0.034905,-0.044435,0.016929,0.040808,0
4,1,4,0,0,1,1,2,2,1,1,...,-0.041606,0.034454,0.060111,-0.052395,0.068730,-0.012220,-0.054667,0.000948,0.034500,0


In [66]:
final_df.to_csv("final_df.csv", index=None)

In [69]:
pd.read_csv("final_df.csv").head()

,0,1,2,3,4,5,6,7,8,9,...,633,634,635,636,637,638,639,640,641,label
0,1,0,0,0,0,0,0,0,0,0,...,0.040750,0.023374,-0.009162,0.054658,0.067103,-0.041042,-0.054814,0.058525,-0.004794,0
1,1,1,0,0,1,1,2,2,1,1,...,-0.099840,0.004646,0.058159,-0.028178,-0.007119,0.028393,-0.038878,0.061012,-0.012936,0
2,1,2,0,0,1,1,2,2,1,1,...,-0.041606,0.034454,0.060111,-0.052395,0.068730,-0.012220,-0.054667,0.000948,0.034500,0
3,1,3,0,0,1,1,2,2,1,1,...,-0.094596,-0.066634,0.036554,0.026686,-0.024640,0.034905,-0.044435,0.016929,0.040808,0
4,1,4,0,0,1,1,2,2,1,1,...,-0.041606,0.034454,0.060111,-0.052395,0.068730,-0.012220,-0.054667,0.000948,0.034500,0
